# Rmeove duplicated species and those not on GBIF

Short script that takes the species checklist and removed any duplicate species entries.

If also removes entries not found on GBIF.

In [ ]:
import pandas as pd
import os

In [ ]:
# Load the species checklist
checklist_name = "thailand-moths"

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+"-keys.csv"))

In [ ]:
# Remove not available species
mask = df["accepted_taxon_key"] != -1
df = df[mask]

In [ ]:
# in most cases the duplication arises due to synonyms so lets order by that
df = df.sort_values(by=['status'])

# Find duplicates
duplicates = df[df.duplicated(subset='accepted_taxon_key', keep=False)]

# Printing duplicate rows
print("Duplicate Rows:")
duplicates

In [ ]:
# Lets inspect cases where duplication is not caused by synonyms
duplicated_no_syn = duplicates.loc[duplicates['status']!='SYNONYM', ]
duplicated_no_syn[duplicated_no_syn.duplicated(subset='accepted_taxon_key', keep=False)]

These are flagged as subspecies in the input file so fine to remove

In [ ]:
# Removing duplicate rows
df_unique = df.drop_duplicates(subset='accepted_taxon_key', keep='first')

In [ ]:
df_unique.shape

In [ ]:
# Save the csv file
df_unique.to_csv(os.path.join("../species_checklists/",checklist_name+"-keys-nodup.csv"), index=False)